# Port Expenses API Example

## Setup
Install the Signal Ocean SDK:
```
pip install signal-ocean
```
Set your subscription key acquired here: https://apis.signalocean.com/profile

In [ ]:
signal_ocean_api_key = '' #replace with your subscription key

## Call the port expenses API
The Port Expenses API retrieves port expenses information.

In [ ]:
from signal_ocean import Connection, PortExpensesAPI
from signal_ocean.port_expenses import Canal, Operation, OperationStatus, \
    EstimationStatus, ItalianAnchorageDues
from datetime import datetime

connection = Connection(api_key=signal_ocean_api_key)
api = PortExpensesAPI(connection)

Get possible canal, operation, operation and estimation status and
italian anchorage dues values, for example:

In [ ]:
Operation.__members__

#### Get required formula parameters
This method will return the parameters that are required in expenses calculation
depending on the port ID and/or the vessel type ID.

In [ ]:
port_id = 3153
req_params = api.get_required_formula_parameters(port_id)

#### Get port expenses
The port expenses can be requested given the IMO and port ID. The optional
parameters can be passed based on the result of get_required_formula_parameters.

In [ ]:
imo = 9867621
port_id = 3153
estimated_time_of_berth = datetime(2020, 2, 27, 17, 48, 11)
port_expenses = api.get_port_expenses(imo, port_id, estimated_time_of_berth=estimated_time_of_berth)

#### Get canal expenses
In order to retrieve canal expenses all the parameters of the corresponding method
are required.

In [ ]:
canal = Canal.SUEZ
imo = 9867621
open_port_id = 3773
load_port_id = 3360
discharge_port_id = 3794
ballast_speed = 12.0
laden_speed = 12.5
operation_status = OperationStatus.LADEN
formula_calculation_date = datetime(2020, 2, 27, 17, 48, 11)
open_date = datetime(1, 1, 1, 0, 0, 0)
load_sail_date = datetime(1, 1, 1, 0, 0, 0)
cargo_type = "Product"
canal_expenses = api.get_canal_expenses(canal, imo, open_port_id, load_port_id, discharge_port_id,
                                        ballast_speed, laden_speed, operation_status,
                                        formula_calculation_date, open_date, load_sail_date, cargo_type)

#### Get port model vessel expenses
In order to retrieve port model vessel expenses, port ID, vessel type ID and the
formula calculation date are required. The rest of parameters are filled in with
the default values if None is passed.

In [ ]:
port_id = 3324
vessel_type_id = 5
formula_calculation_date = datetime(2020, 3, 8, 0, 0, 0)
pmv_expenses = api.get_port_model_vessel_expenses(port_id, vessel_type_id, formula_calculation_date)

#### Get canal model vessel expenses
Similarly to canal expenses, all parameters are required in order to retrieve canal
model vessel expenses.

In [ ]:
canal = Canal.BALTIC
open_port_id = 3773
load_port_id = 3360
discharge_port_id = 3794
operation_status = OperationStatus.BALLAST
formula_calculation_date = datetime(2020, 2, 27, 17, 48, 11)
cmv_expenses = api.get_canal_model_vessel_expenses(canal, open_port_id, load_port_id, discharge_port_id,
                                                    operation_status, formula_calculation_date)